# A data analysis of the NYC taxi rides - part 2

We saw in part 1 that using a static visualisation for a flow, as well as loading the data into memory has its limits.
In this notebook, we will go through the following steps required to generate animations and heat maps to visualise the flow of passengers of the yello taxis in 2018:
- set up the database
- query the data
- create an animation
- create a heat map

As a reminder, we are trying to answer the following questions:
- Can we see trends in the flow of passengers in 2018?
- Is there a difference on holidays, hottest or coldest day of the year?
- Is there a difference between weekdays and weekends?
- Depending on the zone we look at, where are people most likely to come from? To go to? Is it different between weekdays and weekends?

What we are targeting is:
- to have an animation for the whole year 2018, with 1-2 seconds of animation showing the flow of passengers every day (i.e the movement of dots on the map representing passengers going from one point to another)
- to have an animation only for aggregated weekdays and weekends data - which would be represented by 1-2 seconds per week, with either just weekdays or just weekends.
- to have a heat map showing for each zone the difference between the average of *incoming* passengers between weekdays and weekends (so for each zone we have a map, that uses a color code to show where people are mostly coming from, and whether it is more on weekdays or on weekends).
- to have a heat map showing for each zone the difference between the average of *outgoing* passengers between weekdays and weekends (so for each zone we have a map, that uses a color code to show where people are mostly going to, and whether it is more on weekdays or on weekends).

We should be able to identify trends from those visualizations in order to answer the questions.

## Step 1 - Database set up

**Environment details and cleaning steps**


Using MariaDB (5.7.18), we created three tables:
- taxi_rides_2018 (primary key: auto-increment ID)
- taxi_rides_2017 (primary key: increment ID)
- taxi_zone_lookup_table (primary key: LocationID)

As an idea of the size of the table, taxi_rides_2018 has about 97 million rows avec cleanup (about 100M beforehand). 
For both 2017 and 2018, we defined additional indexes to speed up the query:
- PULocationID
- DOLocationID
- Date (column added)
- Weekday (column added)

We decided not to join the zone lookup table with the trips tables, but to do the join when querying the data.

We performed a few cleaning steps:
- Remove rows with values = 0 :
    - PULocationID
    - DOLocationID
    - passenger_count
    - tpep_pickup_datetime
    - tpep_dropff_datetime
- Remove rows with unused values:
    - PULocationID equals to 264 or 265 (unknown zone id)
    - DOLocationID equals to 264 or 265 (unknown zone id)
- Remove rows with negative values:
    - fare_amount
    - extra
    - mta_tax
    - tip_amount
    - tolls_amount
    - improvement_surcharge
- Add a column with only the date (as an index of the table)
- Add a column with the weekday (as an index of the table)


**Queries for table creation and clean up**

In [ ]:
#List of the queries (example of 2017)

#Create a table with all columns and indexes
CREATE TABLE taxi_rides_2017 (
    id INT NOT NULL AUTO_INCREMENT FIRST,
    VendorID INTEGER NOT NULL,
    tpep_pickup_datetime TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    tpep_dropoff_datetime TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    pickup_date DATE NULL,
    pickup_weekday INTEGER NOT NULL,
    dropoff_date DATE NULL,
    dropoff_weekday INTEGER NOT NULL,
    passenger_count INTEGER NULL,
    trip_distance FLOAT NULL,
    RatecodeID INTEGER NOT NULL,
    store_and_fwd_flag CHARACTER(1) NOT NULL,
    PULocationID INTEGER NOT NULL,
    DOLocationID INTEGER NOT NULL,
    payment_type INTEGER NOT NULL,
    fare_amount FLOAT NULL,
    extra FLOAT NULL,
    mta_tax FLOAT NULL,
    tip_amount FLOAT NULL,
    tolls_amount FLOAT NULL,
    improvement_surcharge FLOAT NULL,
    total_amount FLOAT NULL,
    PRIMARY KEY (id),
    INDEX pickup_date (pickup_date),
    INDEX pickup_weekday (pickup_weekday),
    INDEX dropoff_date (dropoff_date),
    INDEX dropoff_weekday (dropoff_weekday),
    FOREIGN KEY (PULocationID) REFERENCES taxi_zone_lookup_table(LocationID),
    FOREIGN KEY (DOLocationID) REFERENCES taxi_zone_lookup_table(LocationID)
);

#Load the data - merged file for a year
LOAD DATA LOCAL INFILE '/Users/acoullandreau/Desktop/Taxi_rides_DS/2017/merged_2017.csv' 
INTO TABLE taxi_rides_2017 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS#Ignore header
(VendorID,tpep_pickup_datetime,tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID,	DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount) 
SET id=null,#sets ID to auto-increment
pickup_date = DATE(tpep_pickup_datetime),
pickup_weekday = WEEKDAY(tpep_pickup_datetime), 
dropoff_date = DATE(tpep_dropoff_datetime), 
dropoff_weekday = WEEKDAY(tpep_dropoff_datetime)
;

#Clean up the data
DELETE FROM nyc_taxi_rides.taxi_rides_2017 
WHERE PULocationID IN (0, 264, 265) 
OR DOLocationID IN (0, 264, 265) 
OR passenger_count  = 0 
OR tpep_pickup_datetime = 0 
OR tpep_dropoff_datetime  = 0 
OR fare_amount <0 
OR extra<0 
OR mta_tax<0 
OR tip_amount<0 
OR tolls_amount<0 
OR improvement_surcharge<0;

Once the database was up and ready, we could start writting the set of functions and the python script that would allow us to connect to the database, query the data, process it and render it, either in the form of an animation (video) or a heat map.


## Step 2 - Building a python script - part 1 (base map and query)

*Note:*

*The code is documented in this notebook, with comments inside each function, as well as on GitHub, where both a documentation and a graph of connection of each function to the other are available. The idea of the graph is to illustrate the logical flow of the script.*

To start with, we will need to:
- process the shapefile
- render a base map
- prepare the query
- connect to the database and execute the query

The functions below can be used to go through each of these steps. 

In [1]:
#Import to be able to run the code below
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import shapefile as shp
from pyproj import Proj, transform
import cv2
import mysql.connector

In [2]:
def shp_to_df(sf):
 
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    
    return df


def calculate_centroid(points):

    x_sum = 0
    y_sum = 0
    for coords in points:
        x_sum += coords[0]
        y_sum += coords[1]
        
    x_mean = x_sum/len(points)
    y_mean = y_sum/len(points)
    
    return x_mean, y_mean


def calculate_boundaries(points):
  
    x_max = -99999999
    x_min = 99999999
    y_max = -99999999
    y_min = 99999999
    
    for coords in points:
        if coords[0] > x_max:
            x_max = coords[0]
        if coords[0] < x_min:
            x_min = coords[0]
        if coords[1] > y_max:
            y_max = coords[1]
        if coords[1] < y_min:
            y_min = coords[1]
        
    max_bound = (x_max, y_max)
    min_bound = (x_min, y_min)
    
    return max_bound, min_bound


def process_shape_boundaries(df_sf, sf):
    
    shape_dict = {}
    index_list = df_sf.index.tolist()
    
    for zone_id in index_list:
        #for each zone id available in the shapefile
        if zone_id not in shape_dict:
            #we only process the coordinates if it is not yet included in the dictionary
            shape_zone = sf.shape(zone_id)
            
            points = [(i[0], i[1]) for i in shape_zone.points]
            
            x_center, y_center = calculate_centroid(points)
            max_bound, min_bound = calculate_boundaries(points)
            
            #we add to the dictionary, for the zone id, the shape boundaries as well
            #as the coordinates of the center of the shape znd the zone extreme boundaries
            shape_dict[zone_id] = {}
            shape_dict[zone_id]['points'] = points
            shape_dict[zone_id]['center'] = (x_center, y_center)
            shape_dict[zone_id]['max_bound'] = max_bound
            shape_dict[zone_id]['min_bound'] = min_bound
            
    return shape_dict  

In [3]:
def find_max_coords(shape_dict):
    
    all_max_bound = []
    all_min_bound = []
    
    for zone in shape_dict:
        zone_shape = shape_dict[zone]
        max_bound_zone = zone_shape['max_bound']
        min_bound_zone = zone_shape['min_bound']
        all_max_bound.append(max_bound_zone)
        all_min_bound.append(min_bound_zone)
    
    map_max_bound, unused_max = calculate_boundaries(all_max_bound)
    unused_min, map_min_bound = calculate_boundaries(all_min_bound)
    
    return map_max_bound, map_min_bound


def define_projection(map_max_bound, map_min_bound, image_size):
    
    #We get the max 'coordinates' for both the target image and the shape we want to draw
    image_x_max = image_size[0]
    image_y_max = image_size[1]
    map_x_max = map_max_bound[0]
    map_y_max = map_max_bound[1]
    map_x_min = map_min_bound[0]
    map_y_min = map_min_bound[1]
    
    projection = {}

    #we check which size is bigger to know based on which axis we want to scale our shape to
    #we do the comparison using the aspect ratio expectations (dividing each axis by the
    #size of the target axis in the new scale)
    if (map_x_max - map_x_min)/image_x_max > (map_y_max - map_y_min)/image_y_max:
        conversion = image_x_max / (map_x_max - map_x_min)
        axis_to_center = 'y'#we store the axis we will want to center on based on which
        #axis we perform the scaling from
    else:
        conversion = image_y_max / (map_y_max - map_y_min)
        axis_to_center = 'x'

    projection['image_size'] = image_size
    projection['map_max_bound'] = map_max_bound
    projection['map_min_bound'] = map_min_bound
    projection['conversion'] = conversion
    projection['axis_to_center'] = axis_to_center
    
    return projection


def convert_projection(x, y, projection, inverse=False):
    
    x_min = projection['map_min_bound'][0]
    y_min = projection['map_min_bound'][1]
    conversion = projection['conversion']
    
    if inverse == False:
        #to be able to center the image, we first translate the coordinates to the origin
        x = (x - x_min) *conversion
        y = (y - y_min) *conversion
    else:
        x = (x + x_min) /conversion
        y = (y + y_min) /conversion
        
    return x, y


def convert_shape_boundaries(zone_shape_dict, projection):
    
    converted_dict = {}
    axis_to_center = projection['axis_to_center']
    image_x_max = projection['image_size'][0]
    image_y_max = projection['image_size'][1]
    map_max_bound_converted = (convert_projection(projection['map_max_bound'][0], projection['map_max_bound'][1], projection))
    map_min_bound_converted = (convert_projection(projection['map_min_bound'][0], projection['map_min_bound'][1], projection))
    
    if axis_to_center == 'x':
        center_translation = (image_x_max - (map_max_bound_converted[0] - map_min_bound_converted[0]))/2
    else:
        center_translation = (image_y_max - (map_max_bound_converted[1] - map_min_bound_converted[1]))/2
    
    
    for zone_id in zone_shape_dict:
        curr_shape = zone_shape_dict[zone_id]
        
        points = curr_shape['points']
        x_center = curr_shape['center'][0]
        y_center = curr_shape['center'][1]
        max_bound = curr_shape['max_bound']
        min_bound = curr_shape['min_bound']
        
        converted_points = []
        for point in points:
            #we convert the coordinates to the new coordinate system
            converted_point = [0, 0] 
            converted_point[0], converted_point[1] = convert_projection(point[0], point[1], projection)
            #we center the map on the axis that was not used to scale the image
            if axis_to_center == 'x':
                converted_point[0] = converted_point[0] + center_translation
            else:
                converted_point[1] = converted_point[1] + center_translation
            
            #we mirror the image to match the axis alignment
            converted_point[1] = image_y_max - converted_point[1]
            converted_points.append(converted_point)
        
        #we convert the center and the max and min boundaries
        x_center, y_center = calculate_centroid(converted_points)
        max_bound = (convert_projection(max_bound[0], max_bound[1], projection))
        min_bound = (convert_projection(min_bound[0], min_bound[1], projection))
        
        
        #We edit the dictionary with the new coordinates
        converted_dict[zone_id] = {}
        converted_dict[zone_id]['points'] = converted_points
        converted_dict[zone_id]['center'] = (x_center, y_center)
        converted_dict[zone_id]['max_bound'] = max_bound
        converted_dict[zone_id]['min_bound'] = min_bound
        
    return converted_dict  

In [4]:
def get_shape_set_to_draw(map_type, shape_dict, df_sf, image_size):

    #we define if we want to draw the whole map or only a borough (in this case map_type
    #should be the borough name)
    if map_type == 'total':
        shape_dict = shape_dict
    else:
        #we select the list of zone_id we want to draw that belong only to the targeted 
        #borough to draw
        shape_dict = reduce_shape_dict_to_borough(shape_dict, df_sf, map_type)
    
    #We define the projection parameters to be able to convert the coordinates into
    #the image scale coordinate system
    #we convert the coordinates of the shapes to draw
    map_max_bound, map_min_bound = find_max_coords(shape_dict)
    projection = define_projection(map_max_bound, map_min_bound, image_size)
    converted_shape_dict = convert_shape_boundaries(shape_dict, projection)
    
    return converted_shape_dict, projection


def reduce_shape_dict_to_borough(shape_dict, df_sf, borough_name):
        
    borough_df = df_sf[df_sf['borough']==borough_name]
    borough_id = []
    for objectid in borough_df.index:
        borough_id.append(objectid)
    
    reduced_shape_dict = {}
    #we add to the reduced_shape_dict only the zones belonging to the borough area targeted
    for zone_id in borough_id:
        reduced_shape_dict[zone_id] = shape_dict[zone_id]
    
    return reduced_shape_dict


def draw_base_map(draw_dict):
    
    #We extract the variables we will need from the input dictionary
    image_size = draw_dict['image_size']
    map_type = draw_dict['map_type']
    title = draw_dict['title']
    shape_dict = draw_dict['shape_dict']
    df_sf = draw_dict['df_sf']
    render_single_borough = draw_dict['render_single_borough']
                    
    #first we create a blank image, on which we will draw the base map
    width = image_size[0]
    height = image_size[1]
    base_map = np.zeros((height,width,3), np.uint8) #Size of the image 1080 height, 1920 width, 3 channels of colour
    base_map[:, :] = [0, 0, 0] #Sets the color to white
    
    #we isolate the set of shapes we want to draw in the right coordinate system
    converted_shape_dict, projection = get_shape_set_to_draw(map_type, shape_dict, df_sf, image_size)
    
    if render_single_borough == False:
        #we use the projection parameters from the borough we want to focus on
        #we calculate the coordinates for the whole map
        converted_shape_dict = convert_shape_boundaries(shape_dict, projection)
        
    #we draw each shape of the dictionary on the blank image, 
    #either the full map or only a borough 
    for item in converted_shape_dict:
        shape = converted_shape_dict[item]
        points = shape['points']
        pts = np.array(points, np.int32)
        cv2.polylines(base_map, [pts], True, (255, 255, 255), 1, cv2.LINE_AA)

    #we display general text information   
    display_general_information_text(base_map, map_type, title)
    
    return base_map, projection



In [5]:
def prepare_sql_query(query_dict):

    #We extract the variables we will need from the input dictionary
    data_table = query_dict['data_table']
    lookup_table = query_dict['lookup_table']
    aggregated_result = query_dict['aggregated_result']
    date = query_dict['date']
    filter_query_on_borough = query_dict['filter_query_on_borough']
    
    #first we synthesise what we want to fetch
    if aggregated_result == 'count':
        aggregated_result = 'COUNT(passenger_count)'
    elif aggregated_result == 'avg':
        aggregated_result = 'AVG(passenger_count)'
    
    #then we work on the 'WHERE' statements and the JOIN 
    if filter_query_on_borough != False:
        query = ("SELECT pu_id, do_id, aggregated_result FROM (\
                    SELECT PULocationID pu_id, DOLocationID do_id, {0} aggregated_result\
                    FROM {1} tr_2018\
                    WHERE pickup_date = '{2}'\
                    GROUP BY pu_id, do_id\
                    ORDER by aggregated_result\
                ) AS tr_2018\
                 JOIN {3} lookup_pu\
                 ON lookup_pu.LocationID = tr_2018.pu_id \
                 JOIN {3} lookup_do \
                 ON lookup_do.LocationID = tr_2018.do_id \
                 WHERE lookup_pu.borough_name = '{4}' AND lookup_do.borough_name = '{4}'".format
                (aggregated_result, data_table, date, lookup_table, filter_query_on_borough))

    else:
        query = ("SELECT PULocationID pu_id, DOLocationID do_id, {0} aggregated_result\
                    FROM {1} AS tr_2018\
                    WHERE pickup_date = '{2}'\
                    GROUP BY pu_id, do_id".format(aggregated_result, data_table, date))


    return query



In [6]:
def make_sql_query(query, database):

    #connect to the database
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="dllpsax00",
        database=database
        )

    #execute the query...
    cursor = db.cursor()
    cursor.execute(query)

    # ...and store the output
    results=[]
    for result in cursor:
        results.append(result)

    cursor.close()

    return results

## Step 3 - Building a python script - part 2 (animation)

Once we havea base map and the result of the query, we are able to start processing the data to render it in an animation.

We will need to:
- process the query results
- render each point by interpolating its position
- render each frame
- render the animation

The functions below can be used to go through each of these steps.

In [48]:
def interpolate_next_position(origin_coords, destination_coords, tot_frames, curr_frame):
        
    #as to perform the arithmetic operations, we convert everything to float for more 
    #precision
    x_origin = float(origin_coords[0])
    y_origin = float(origin_coords[1])
    x_destination = float(destination_coords[0])
    y_destination = float(destination_coords[1])
    tot_frames = float(tot_frames - 1)
    curr_frame = float(curr_frame)
    
    delta_x = (x_destination - x_origin)/tot_frames
    delta_y = (y_destination - y_origin)/tot_frames
    
    #the rendering with OpenCV demands integers values for the positioning, so we convert
    #w and y to int
    new_x = int(x_origin+delta_x*curr_frame)
    new_y = int(y_origin+delta_y*curr_frame)
    
    return new_x, new_y



def render_point_on_map(x_point, y_point, weight, base_map, colour):
      
    cv2.circle(base_map, (x_point,y_point), weight, colour, -1)
    
    
def convert_id_shape(idx, inverse = False):
        
    if inverse == False:
        idx = idx - 1
    else:
        idx = idx + 1
    
    return idx
    

def build_query_dict(render_animation_dict):
    
    #First, we extract the variables we will need from the input dictionary
    time_granularity = render_animation_dict['time_granularity']
    data_table = render_animation_dict['data_table']
    lookup_table = render_animation_dict['lookup_table']
    aggregated_result = render_animation_dict['aggregated_result']
    filter_query_on_borough = render_animation_dict['filter_query_on_borough']
    
    #we instantiate the query_dict and start filling it with query parameters
    query_dict = {}
    query_dict['data_table'] = data_table
    query_dict['lookup_table'] = lookup_table
    query_dict['aggregated_result'] = aggregated_result
    
    #we handle the borough related WHEN statement
    if filter_query_on_borough == False:
        query_dict['filter_query_on_borough'] = False
    else:
        query_dict['filter_query_on_borough'] = filter_query_on_borough
    
    #we handle the time related WHEN statements
    period = render_animation_dict['period']
    start_date = period[0]
    end_date = period[1]
        
    if start_date == end_date:
        query_dict['date'] = start_date

    else:
        #if the period is more than one date, we will have to loop through the
        #date range and render multiple series of 60 frames (1 second at 60 fps per day)
        #Thus the loop needs to be handled by the main plotting function, and here we
        #simply add a flag to the query dict that will be transformed by the plotting
        #function
        query_dict['date'] = 'loop_through_period'

    #used specifically for the animation logic
    if time_granularity == 'specific_weekdays':
        specific_weekdays = render_animation_dict['weekdays']
        query_dict['specific_weekdays'] = 'on_specific_weekdays'
    
    #used specifically for the animation logic
    elif time_granularity == 'period':
        query_dict['specific_weekdays'] = False
    
    #used specifically for the heat_map logic
    elif time_granularity == 'weekdays_vs_weekends':
        query_dict['specific_weekdays'] = 'weekdays_vs_weekends'
    
    return query_dict
    


In [8]:
def compute_weight(map_type, weight, max_passenger):
    #we normalise the weight of the point based on the max number of passengers
    #which means that from one day to another, although the biggest point will have the
    #same size, it will not represent the same number of passengers (compromise to
    #prevent having huge differences between the points, or squishing too much the scale
    #by using a log). 
    
    if map_type != 'total':
        weight = weight/max_passenger*20
    else:
        weight = weight/max_passenger*10

    weight = int(weight)

    return weight


def display_specific_text(rendered_frame, date, map_type, min_pass, max_pass):
    #note that these position are based on an image size of [1920, 1080]
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    #64, 64, 64
    #displays the date and the weekday, and if it is a special date
    cv2.putText(rendered_frame, date, (40, 150), font, 1.3, (221, 221, 221), 1, cv2.LINE_AA)
    
    special_dates_2018 = {'2018-01-01':'New Year', '2018-12-25':'Christmas',
                         '2018-02-14':'Valentine\'s Day', '2018-07-04':'National Day',
                         '2018-07-01':'Hottest Day', '2018-01-07':'Coldest Day'}
    if date in special_dates_2018:
        cv2.putText(rendered_frame, special_dates_2018[date], (40, 200), font, 1.3, (221, 221, 221), 1, cv2.LINE_AA)
        
    date_timestamp = pd.Timestamp(date)
    weekday = date_timestamp.dayofweek
    weekdays = {0:'Monday', 1:'Tuesday',2:'Wednesday',3:'Thursday', 
                4:'Friday', 5:'Saturday', 6:'Sunday'}
    weekday = weekdays[weekday]
    cv2.putText(rendered_frame, weekday, (40, 95), font, 1.3, (221, 221, 221), 1, cv2.LINE_AA)

    #displays the legend of the size of the circles
    cv2.putText(rendered_frame, 'Number of passengers', (35, 380), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
    max_weight = compute_weight(map_type, max_pass, max_pass)
    cv2.circle(rendered_frame, (40, 420), max_weight, (255, 255, 255), 1)
    cv2.putText(rendered_frame, '{} passengers'.format(max_pass), (80, 420), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    min_weight = compute_weight(map_type, min_pass, max_pass)
    cv2.circle(rendered_frame, (40, 460), min_weight, (255, 255, 255), 1)
    cv2.putText(rendered_frame, '{} passenger'.format(min_pass), (80, 460), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    

def display_general_information_text(image, map_type, video_title):

    #note that these position are based on an image size of [1920, 1080]
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    #displays the name of the boroughs of the city
    if map_type == 'total':
        #name of borough Manhattan
        cv2.putText(image, 'Manhattan', (770, 360), 
            font, 0.8, (255, 255, 255), 1, cv2.LINE_AA) 
        #name of borough Brooklyn
        cv2.putText(image, 'Brooklyn', (1130, 945), 
            font, 0.8,(255, 255, 255), 1, cv2.LINE_AA) 
        #name of borough Staten Island
        cv2.putText(image, 'Staten Island', (595, 1030), 
            font, 0.8, (255, 255, 255), 1, cv2.LINE_AA) 
        #name of borough Queens
        cv2.putText(image, 'Queens', (1480, 590), 
            font, 0.8, (255, 255, 255), 1, cv2.LINE_AA) 
        #name of borough Bronx
        cv2.putText(image, 'Bronx', (1370, 195), 
            font, 0.8, (255, 255, 255), 1, cv2.LINE_AA) 
    
    else:
        video_title = video_title + ' in ' + map_type

    #displays the legend of the colour code
    cv2.putText(image, 'Origin and destination', (35, 260), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.circle(image, (40, 290), 10, (141, 91, 67), -1)
    cv2.putText(image, 'Identical', (60, 300), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.circle(image, (40, 320), 10, (135,162,34), -1)
    cv2.putText(image, 'Distinct', (60, 330), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    
    #displays the title of the video
    #cv2.putText(image, video_title, (500, 1050), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

    

def render_frame(frame, base_map, query_results, converted_shape_dict, map_type):

        
    #we make a copy of the map on which we will render the frame (each frame being
    #rendered on a new copy)
    map_rendered = base_map.copy()

    #we get the value of the min and max number of passengers from the query result
    min_passenger_itinerary = min(query_results, key=lambda x:x[2])
    max_passenger_itinerary = max(query_results, key=lambda x:x[2])
    max_passenger = max_passenger_itinerary[2]
    min_passenger = min_passenger_itinerary[2]
    
    #we get each tuple from the query result, in the form (origin_id, dest_id, weight)
    for itinerary in query_results:
        zone_id_origin = convert_id_shape(itinerary[0])
        zone_id_destination = convert_id_shape(itinerary[1])

        weight = itinerary[2]
        weight = compute_weight(map_type, weight, max_passenger)

        #we get the coordinates of the center of the origin and the destination
        origin_coords = converted_shape_dict[zone_id_origin]['center']
        destination_coords = converted_shape_dict[zone_id_destination]['center']

        if frame == 0:
            #we start the rendering with the point at the origin
            #we convert to int as to be able to plot the point with opencv
            coords_point_to_draw = (int(origin_coords[0]), int(origin_coords[1]))

        else:
            #we extrapolate the position of the point between the origin and the
            #destination, as to have the point move from origin to destination
            #in 60 frames
            coords_point_to_draw = interpolate_next_position(origin_coords, destination_coords, 60, frame)

        x_point = coords_point_to_draw[0]
        y_point = coords_point_to_draw[1]

        if zone_id_origin == zone_id_destination:
            colour = (141, 91, 67)
        else:
            colour = (135,162,34)

        render_point_on_map(x_point, y_point, weight, map_rendered, colour)

    return map_rendered, min_passenger, max_passenger


def render_all_frames(render_frame_dict):
    
    #we extract the arguments we need from the input dictionary
    query_dict = render_frame_dict['query_dict']
    database = render_frame_dict['database']
    base_map = render_frame_dict['base_map']
    converted_shape_dict = render_frame_dict['converted_shape_dict']
    map_type = render_frame_dict['map_type']
    frames = render_frame_dict['frames']
    
    #we query the database
    query = prepare_sql_query(query_dict)
    query_results = make_sql_query(query, database)
    
    #we use the results of the query to render 60 frames
    #we want to render an animation of 1 second per given date, at 60 fps. 
    for frame in range(0, 60):  
        rendered_frame, min_pass, max_pass = render_frame(frame, base_map, query_results, 
                                                        converted_shape_dict, map_type)       
        
        #we display frame related text
        display_specific_text(rendered_frame, query_dict['date'], map_type, min_pass, max_pass)
        
        frames.append(rendered_frame)

    return frames


def make_video_animation(frames, image_size, map_type):
        
    #Build the title for the animation
    if map_type == 'total':
        title = 'Animation_{}.avi'.format('NYC')
    else:
        title = 'Animation_{}.avi'.format(map_type)
    
    animation = cv2.VideoWriter(title, cv2.VideoWriter_fourcc(*'DIVX'), 30, image_size)
    #video title, codec, fps, frame size

    for i in range(len(frames)):
        animation.write(frames[i])
    
    animation.release()
    

In [49]:
def render_animation_query_output(render_animation_dict):
    
    #We extract the variables we will need from the input dictionary
    query_dict = render_animation_dict['query_dict']
    base_map = render_animation_dict['base_map']
    map_type = render_animation_dict['map_type']
    shape_dict = render_animation_dict['shape_dict']
    df_sf = render_animation_dict['df_sf']
    database = render_animation_dict['database']
    image_size = render_animation_dict['image_size']
    render_single_borough = render_animation_dict['render_single_borough']
    
    if query_dict['filter_query_on_borough'] == False:
        #in this case, we may want the base map to be reduced to map_type, but the query
        #to be performed on the whole city - thus we want to represent points that may
        #not be inside the shape of the reduced base map
        projection = render_animation_dict['projection']
        converted_shape_dict = convert_shape_boundaries(shape_dict, projection)
    
    else:
        #we isolate the set of zones we want to draw points for in the right coordinate system
        converted_shape_dict, projection = get_shape_set_to_draw(map_type, shape_dict, df_sf, image_size)


    #we build a dictionary for the details of the rendering of each frame
    render_frame_dict = {'query_dict':query_dict,'database':database,
                         'base_map':base_map,'converted_shape_dict':converted_shape_dict,
                         'map_type':map_type,'frames':[]}
    
    #we render frames depending on the results of the query and the period inputted
    
    if query_dict['date'] == 'loop_through_period':
        #if we have the flag loop_through_period in the query dict, it means the period
        #set for the query is multiple dates, therefore we need to render multiple times
        #60 frames
        period = render_animation_dict['period']
        daterange = pd.date_range(period[0],period[1])
        #we run queries for each date in the daterange specified
        for single_date in daterange: 
            date = pd.to_datetime(single_date)
            if query_dict['specific_weekdays'] == 'on_specific_weekdays':
                weekdays = render_animation_dict['weekdays']
                
                #we check if the date of the daterange matches the weekday(s) we target
                if date_timestamp.dayofweek in weekdays:
                    single_date = date.date().strftime('%Y-%m-%d')
                    query_dict['date'] = single_date
                    frames = render_all_frames(render_frame_dict)
                    render_frame_dict['frames'] = frames
                    
                else:
                    #if a date in the range is not among the weekdays we want, we skip it
                    continue
            else:
                single_date = date.date().strftime('%Y-%m-%d')
                query_dict['date'] = single_date
                frames = render_all_frames(render_frame_dict)
                render_frame_dict['frames'] = frames
    
    else:
        #we have a single period (i.e. one single day) to render results for and we 
        #just render 60 frames for this period
        #just in case we check that there is no mismatch between the single day and the
        #argument containing specific weekdays restrictions if any
        
        if query_dict['specific_weekdays'] == 'on_specific_weekdays':
                weekdays = render_animation_dict['weekdays']
                
                #we check if the date of the daterange matches the weekday(s) we target
                date = pd.Timestamp(query_dict['date'])
                
                if date.dayofweek in weekdays:
                    frames = render_all_frames(render_frame_dict)
        
                else:
                    print("The date selected does not match the weekday(s) indicated. "
                        "Please select either an interval ('time_granularity':'period') "
                        "or a valid weekday(s) list.")
        
        else:
            frames = render_all_frames(render_frame_dict)
    
    if map_type == 'total':
        print('Rendering the results for NYC...')
    else:
        print('Rendering the results for {}...'.format(map_type))
    
    #we compile the video from all frames
    make_video_animation(frames, image_size, map_type)
    

    if map_type == 'total':
        print('The video for NYC has been rendered')
    else:
        print('The video for {} has been rendered'.format(map_type))
    
    
    

**Python script**

We can the define the python script that will rely on all functions provided above.

In [17]:
def make_flow_animation(animation_dict):
    #we extract the variables from the input dictionary
    shp_path = animation_dict['shp_path']
    image_size = animation_dict['image_size']
    map_to_render = animation_dict['map_to_render']
    render_single_borough = animation_dict['render_single_borough']
    title = animation_dict['title']
    database = animation_dict['db']
    data_table = animation_dict['data_table']
    lookup_table = animation_dict['lookup_table']
    aggregated_result = animation_dict['aggregated_result']
    filter_query_on_borough = animation_dict['filter_query_on_borough']
    time_granularity = animation_dict['time_granularity']
    period = animation_dict['period']
    weekdays = animation_dict['weekdays']
    
    #First import the shapefile and build the boundaries dictionary
    print('Building the base map...')
    shp_path = shp_path
    sf_nyc = shp.Reader(shp_path)
    df_sf = shp_to_df(sf_nyc)
    shape_boundaries = process_shape_boundaries(df_sf, sf_nyc)
    
    #optional fool_proof check
    #if filter on borough is not False, then it contains the name of a borouhgh, 
    #that happens to be the only one we want to use to draw the base map
    #so we ignore the input of the user in the map_to_render argument
    if filter_query_on_borough !=False:
        map_to_render = [filter_query_on_borough]
    
    #Draw the base map and keep it in a saved variable
    base_maps = []
    if len(map_to_render) == 1:
        map_type = map_to_render[0]
        #we want to render on a single map
        draw_dict = {'image_size':image_size, 'render_single_borough':render_single_borough, 
                     'map_type':map_type, 'title':title, 
                     'shape_dict':shape_boundaries, 'df_sf':df_sf}
        base_map, projection = draw_base_map(draw_dict)
        base_maps.append((map_type, base_map, projection))
    
    else:
        #we want to render multiple animations at once, for different base maps
        for single_map in map_to_render:
            map_type = single_map
            draw_dict = {'image_size':image_size, 'render_single_borough':render_single_borough,
                         'map_type':map_type, 'title':title, 
                         'shape_dict':shape_boundaries, 'df_sf':df_sf}
            base_map, projection = draw_base_map(draw_dict)
            base_maps.append((map_type, base_map, projection))
    
    #we define the render_animation_dict
    render_animation_dict = {'time_granularity':time_granularity, 'period':period,  
                             'weekdays':weekdays,'filter_query_on_borough':filter_query_on_borough, 
                             'image_size':image_size,'shape_dict':shape_boundaries, 
                             'df_sf':df_sf,'database':database, 'data_table':data_table, 
                             'lookup_table':lookup_table,'aggregated_result':aggregated_result,
                             'render_single_borough':render_single_borough,
                             'video_title':title}

    print('Querying the dabase...')
         
    query_dict = build_query_dict(render_animation_dict)
    render_animation_dict['query_dict'] = query_dict
    
    #we render the animation!
    for map_type, base_map, projection in base_maps:
        #we add variabled to the render frame dictionary
        render_animation_dict['base_map'] = base_map
        render_animation_dict['projection'] = projection
        render_animation_dict['map_type'] = map_type

        render_animation_query_output(render_animation_dict)

    
    

This script uses as an input a dictionary, which structure is as follows:

animation_dict = {'shp_path':shp_path, 'image_size':(1920,1080), 
                  'map_to_render':['total', 'Manhattan'],'render_single_borough':False,
                  'filter_query_on_borough':False,
                  'title':'General flow of passengers in 2018', 
                  'db':'nyc_taxi_rides', 'data_table':'taxi_rides_2018',
                 'lookup_table':'taxi_zone_lookup_table', 'aggregated_result':'count',
                 'time_granularity':'period', 'period':['2018-01-01','2018-01-03'],  
                 'weekdays':[]}
                 
**Arguments:**
- shp_path: the path to the shapefile used to render the base map
- image_size: the size of each frame [width, height]
- map_to_render: the base map(s) we want animations for. Always provided as a list. If more than one item is in the list, one animation per item will be rendered.
- render_single_borough: flag to indicate whether we want to focus on a single borough and render *only* the borough (in this case True), or if we simply want to center and zoom on a borough but still render the rest of the map (in this case False)
- filter_query_on_borough: whether we want to execute the query filtering on a borough, or if we want the results for the whole city
- title: the title to display in the animation
- db: the name of the database to connect to
- data_table: the table in which to fetch the data (in our case, the table in which we have the data for 2018)
- lookup_table: the taxi zone lookup table, to match a zone id with the name of a borough
- aggregated_result: the type of result we want from the query, either avg or count (note that the query results will always be structured 'PULocationID', 'DOLocationID', aggregated_result). 
- time_granularity: if we want to filter for specific weekdays or we want results for every day in the provided period
- period: the time interval to consider for the query. If we want for a single date, start and end date should be inputted the same.
- weekdays: the index of the weekday(s) we want data for (0 being Monday, 6 being Sunday). If we want to filter on one or more weekday, time_granularity should be set to 'on_specific_weekdays'. If we we do not want to filter on any weekday, time_granularity should be set to 'period' and the array of weekdays left empty []. 


In [22]:
animation_dict = {'shp_path':shp_path, 'image_size':(1920,1080), 
                  'map_to_render':['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'],
                  'render_single_borough':False,
                  'filter_query_on_borough':False,
                  'title':'General flow of passengers on weekdays in 2018', 
                  'db':'nyc_taxi_rides', 'data_table':'taxi_rides_2018',
                 'lookup_table':'taxi_zone_lookup_table', 'aggregated_result':'count',
                 'time_granularity':'period', 'period':['2018-01-01','2018-01-01'],  
                 'weekdays':[0, 1, 2, 3, 4]}

make_flow_animation(animation_dict)

Building the base map...
Querying the dabase...
Rendering the results for NYC...
The video for NYC has been rendered
Rendering the results for Manhattan...
The video for Manhattan has been rendered
Rendering the results for Bronx...
The video for Bronx has been rendered
Rendering the results for Queens...
The video for Queens has been rendered
Rendering the results for Staten Island...
The video for Staten Island has been rendered
Rendering the results for Brooklyn...
The video for Brooklyn has been rendered


## Step 4 - Building a python script - part 2 (heat maps)

Luckily for this second rendering option, there is a lot we can reuse from the animation logic. The idea of this second visualisation is to render four heat maps per zone:
- incoming flow
- outgoing flow
- difference in incoming flow between weekdays and weekends
- difference in outgoing flow beteen weekdays and weekends

Note that we are going to work with average or count of the number of passengers on the period selected by the user. So say we want the map for 2018 with the aggregated_result set to avg, we are going to average the total number of passenger in 2018, splitted per incoming and outgoing flow. If aggregated_result is set to count, we will simply sum the counts over the year.
Likewise for the comparison of the weekdays and weekends, we will average the number of passengers of weekdays on the whole period, and subtract to it the number of passengers on weekends. Which means that if we have a positive value, there are more people traveling on weekdays, and if we have a negative value, there are more people traveling on weekends. 


What we will want to do is:
- render a base map: either for the whole city, or solely for the borough in which we have the zone we are focusing on.
- make four queries
    - average on the period of the number of passengers grouped by *pick up* location ID -> we will look at *outgoing* flow
    - average on the period of the number of passengers grouped by *drop off* location ID -> we will look at *incoming* flow
    - difference between the average on the period on weekdays and weekends of the number of passengers grouped by *pick up* location ID -> we will look at average difference of *outgoing* flow between weekdays and weekends 
    - difference between the average on the period on weekdays and weekends of the number of passengers grouped by *drop off* location ID -> we will look at average difference of *incoming* flow between weekdays and weekends 
- draw the result maps

As a result, we are going to render quite a lot of map! (5 maps per zone (one focused on the borough, one focused on the zone), 263 zones at most, so more than 1000 maps!). To make browsing easier, we will make the maps available on a web page with a dropdown list to select and display the desired map.

Let's define the missing functions (those specific to the rendering oh the heat maps). 

In [67]:
def prepare_heat_map_sql_query(query_dict):

    {'data_table': 'taxi_rides_2018', 
     'lookup_table': 'taxi_zone_lookup_table', 
     'aggregated_result': 'avg', 
     'filter_query_on_borough': False, 
     'date': ['2018-01-01', '2018-01-02'], 
     'specific_weekdays': 'weekdays_vs_weekends'}
    
    #We extract the variables we will need from the input dictionary
    data_table = query_dict['data_table']
    lookup_table = query_dict['lookup_table']
    aggregated_result = query_dict['aggregated_result']
    date = query_dict['date']
    filter_query_on_borough = query_dict['filter_query_on_borough']
    weekdays_vs_weekends = query_dict['weekdays_vs_weekends']
     
    #first we synthesise what we want to fetch
    if aggregated_result == 'count':
        #we will want to return the sum of count on the period
        aggregated_result = 'COUNT(passenger_count)'
    elif aggregated_result == 'avg':
        #we will want to return the average of count on the period
        aggregated_result = '(COUNT(passenger_count)'
    
    #we prepare the period statements
    if weekdays_vs_weekends == 'weekdays_vs_weekends':
        #in this case we want to query 'separataly' the values in weekdays and weekends
        #and make a difference on the average of the aggregated_result on the period
        
    if type(date) == str:
        #in this case, we want the result on a single day
    
    else:
        #we provided a time interval we want the average of the aggregated_result on the
        #period
        
    
    #avg on total period without borough filter
    query = ("SELECT pu_id, do_id, AVG(aggregated_result) \
             FROM \
                 (SELECT PULocationID pu_id, DOLocationID do_id, \
                         pickup_date date, COUNT(passenger_count) {0} \
                 FROM {1} AS tr_2018 \
                 WHERE pickup_date BETWEEN {2} AND {3} \
                 GROUP BY pu_id, do_id, date) as tab_1 \
             GROUP BY pu_id, do_id").format(aggregated_result, data_table, start_date, 
                                            end_date)
    
    
    #avg on total period with borough filter
    query = ("SELECT pu_id, do_id, AVG(aggregated_result) \
            FROM \
                 (SELECT PULocationID pu_id, DOLocationID do_id, \
                         pickup_date date, COUNT(passenger_count) {0} \
                 FROM {1} AS tr_2018 \
                 WHERE pickup_date BETWEEN {2} AND {3} \
                 GROUP BY pu_id, do_id, date) as tab_1 \
            JOIN {3} lookup_pu\
            ON lookup_pu.LocationID = tab_1.pu_id \
            JOIN {3} lookup_do \
            ON lookup_do.LocationID = tab_1.do_id \
            WHERE lookup_pu.borough_name = '{5}' AND lookup_do.borough_name = '{5}'\
            GROUP BY pu_id, do_id").format(aggregated_result, data_table, start_date, 
                                            end_date, lookup_table, filter_query_on_borough)
    
    
    #diff avg on total period between weekdays and weekends
    query = ("SELECT pu_id, do_id, AVG(aggregated_result) \
             FROM \
                 (SELECT PULocationID pu_id, DOLocationID do_id, \
                         pickup_date date, COUNT(passenger_count) {0} \
                 FROM {1} AS tr_2018 \
                 WHERE pickup_date BETWEEN {2} AND {3} AND pickup_weekday IN (0, 1, 2, 3, 4) \
                 GROUP BY pu_id, do_id, date) as weekday\
             JOIN (SELECT PULocationID pu_id, DOLocationID do_id, \
                 pickup_date date, COUNT(passenger_count) {0} \
                 FROM {1} AS tr_2018 \
                 WHERE pickup_date BETWEEN {2} AND {3} AND pickup_weekday IN (5, 6) \
                 GROUP BY pu_id, do_id, date) as weekend\
             \
             GROUP BY do_id, pu_id").format(aggregated_result, data_table, start_date, 
                                            end_date)
    
    
    
    
    
    
    
    
    
    #avg on total period for incoming
    query = ("SELECT pu_id, do_id, AVG(aggregated_result) \
             FROM \
                 (SELECT PULocationID pu_id, DOLocationID do_id, \
                         pickup_date date, COUNT(passenger_count) aggregated_result \
                 FROM taxi_rides_2018 AS tr_2018 \
                 WHERE pickup_date BETWEEN '2018-01-01 00:00:00' AND '2018-01-02 00:00:00' \
                 GROUP BY pu_id, do_id, date) as tab_1 \
                 
             
             GROUP BY pu_id, do_id")
    
    #then we work on the 'WHERE' statements and the JOIN 
    if filter_query_on_borough != False:
        query = ("SELECT pu_id, do_id, aggregated_result \
                 FROM (\
                    SELECT PULocationID pu_id, DOLocationID do_id, {0} aggregated_result\
                    FROM {1} tr_2018\
                    WHERE pickup_date = '{2}'\
                    GROUP BY pu_id, do_id\
                    ORDER by aggregated_result) AS tr_2018\
                 JOIN {3} lookup_pu\
                 ON lookup_pu.LocationID = tr_2018.pu_id \
                 JOIN {3} lookup_do \
                 ON lookup_do.LocationID = tr_2018.do_id \
                 WHERE lookup_pu.borough_name = '{4}' AND lookup_do.borough_name = '{4}'".format
                (aggregated_result, data_table, date, lookup_table, filter_query_on_borough))

    else:
        query = ("SELECT PULocationID pu_id, DOLocationID do_id, {0} aggregated_result\
                    FROM {1} AS tr_2018\
                    WHERE pickup_date = '{2}'\
                    GROUP BY pu_id, do_id".format(aggregated_result, data_table, date))


    return query

IndentationError: expected an indented block (<ipython-input-67-5a3b69f4f79a>, line 31)

In [64]:
def render_heat_map_query_output(render_heat_map_dict):
    
    #We extract the variables we will need from the input dictionary
    query_dict = render_heat_map_dict['query_dict']
    base_map = render_heat_map_dict['base_map']
    map_type = render_heat_map_dict['map_type']
    shape_dict = render_heat_map_dict['shape_dict']
    df_sf = render_heat_map_dict['df_sf']
    database = render_heat_map_dict['database']
    image_size = render_heat_map_dict['image_size']


    if query_dict['filter_query_on_borough'] == False:
        #in this case, we may want the base map to be reduced to map_type, but the query
        #to be performed on the whole city - thus we want to represent points that may
        #not be inside the shape of the reduced base map
        projection = render_heat_map_dict['projection']
        converted_shape_dict = convert_shape_boundaries(shape_dict, projection)
    
    else:
        #we isolate the set of zones we want to draw points for in the right coordinate system
        converted_shape_dict, projection = get_shape_set_to_draw(map_type, shape_dict, df_sf, image_size)


    #we build a dictionary for the details of the rendering of each frame
    render_map_dict = {'query_dict':query_dict,'database':database,'base_map':base_map,
                       'converted_shape_dict':converted_shape_dict,'map_type':map_type}
    
    if query_dict['date'] == 'loop_through_period':
        #if we have the flag loop_through_period in the query dict, it means the period
        #set for the query is multiple dates, therefore we want the query to return an
        #average on a time interval, and not on a single date
        period = render_heat_map_dict['period']
        daterange = pd.date_range(period[0],period[1])
        query_dict['date'] = period
        
        
        if query_dict['specific_weekdays'] == 'weekdays_vs_weekends':
            return None
            #we query the difference between the average on weekdays and weekends
            #on the whole period
            #query grouped by pu_id
            #query grouped by do_id
        
        return None       
    print(query_dict)
           
        #we query and average on the whole period
        #query grouped by pu_id
        #query grouped by do_id
        

            
 

In [ ]:
        
        date = pd.to_datetime(single_date)
        single_date = date.date().strftime('%Y-%m-%d')
        query_dict['date'] = single_date
            
            
#we run queries for each date in the daterange specified
        for single_date in daterange: 
            date = pd.to_datetime(single_date)
            if query_dict['specific_weekdays'] == 'on_specific_weekdays':
                weekdays = render_animation_dict['weekdays']
                
                #we check if the date of the daterange matches the weekday(s) we target
                if date_timestamp.dayofweek in weekdays:
                    single_date = date.date().strftime('%Y-%m-%d')
                    query_dict['date'] = single_date
                    frames = render_all_frames(render_frame_dict)
                    render_frame_dict['frames'] = frames
                    
                else:
                    #if a date in the range is not among the weekdays we want, we skip it
                    continue
            else:
                single_date = date.date().strftime('%Y-%m-%d')
                query_dict['date'] = single_date
                frames = render_all_frames(render_frame_dict)
                render_frame_dict['frames'] = frames
    
    else:
        #we have a single period (i.e. one single day) to render results for and we 
        #just render 60 frames for this period
        #just in case we check that there is no mismatch between the single day and the
        #argument containing specific weekdays restrictions if any
        
        if query_dict['specific_weekdays'] == 'on_specific_weekdays':
                weekdays = render_animation_dict['weekdays']
                
                #we check if the date of the daterange matches the weekday(s) we target
                date = pd.Timestamp(query_dict['date'])
                
                if date.dayofweek in weekdays:
                    frames = render_all_frames(render_frame_dict)
        
                else:
                    print("The date selected does not match the weekday(s) indicated. "
                        "Please select either an interval ('time_granularity':'period') "
                        "or a valid weekday(s) list.")
        
        else:
            frames = render_all_frames(render_frame_dict)
    
    if map_type == 'total':
        print('Rendering the results for NYC...')
    else:
        print('Rendering the results for {}...'.format(map_type))
    
    #we compile the video from all frames
    make_video_animation(frames, image_size, map_type)
    

    if map_type == 'total':
        print('The video for NYC has been rendered')
    else:
        print('The video for {} has been rendered'.format(map_type))

In [65]:
def make_heat_map(heat_map_dict):
    
    #we extract the variables from the input dictionary
    shp_path = heat_map_dict['shp_path']
    image_size = heat_map_dict['image_size']
    map_to_render = heat_map_dict['map_to_render']
    render_single_borough = heat_map_dict['render_single_borough']
    title = heat_map_dict['title']
    database = heat_map_dict['db']
    data_table = heat_map_dict['data_table']
    lookup_table = heat_map_dict['lookup_table']
    aggregated_result = heat_map_dict['aggregated_result']
    filter_query_on_borough = heat_map_dict['filter_query_on_borough']
    period = heat_map_dict['period']
    
    if heat_map_dict['weekdays_vs_weekends']==True:
        time_granularity = 'weekdays_vs_weekends'
    else:
        time_granularity = 'period'
    
    #First import the shapefile and build the boundaries dictionary
    print('Building the base map...')
    shp_path = shp_path
    sf_nyc = shp.Reader(shp_path)
    df_sf = shp_to_df(sf_nyc)
    shape_boundaries = process_shape_boundaries(df_sf, sf_nyc)
    
    #fool_proof check
    if render_single_borough == True:
        #in this case we do not want to render the whole map of NYC, but only boroughs maps
        if 'total' in map_to_render: 
            #so we remove 'total' from the list of maps to render if it was added by the user
            map_to_render.remove('total')
            
    
    #Draw the base map and keep it in a saved variable
    base_maps = []
    if len(map_to_render) == 1:
        map_type = map_to_render[0]
        #we want to render on a single map
        draw_dict = {'image_size':image_size, 'render_single_borough':render_single_borough, 
                     'map_type':map_type, 'title':title, 
                     'shape_dict':shape_boundaries, 'df_sf':df_sf}
        base_map, projection = draw_base_map(draw_dict)
        base_maps.append((map_type, base_map, projection))
    
    else:
        #we want to render multiple heat maps at once, for different base maps
        for single_map in map_to_render:
            map_type = single_map
            draw_dict = {'image_size':image_size, 'render_single_borough':render_single_borough,
                         'map_type':map_type, 'title':title, 
                         'shape_dict':shape_boundaries, 'df_sf':df_sf}
            base_map, projection = draw_base_map(draw_dict)
            base_maps.append((map_type, base_map, projection))
    
    
    #we define the render_heat_map_dict    
    render_heat_map_dict = {'time_granularity':time_granularity, 'period':period,  
                             'image_size':image_size,'shape_dict':shape_boundaries, 
                             'df_sf':df_sf,'database':database, 'data_table':data_table, 
                             'lookup_table':lookup_table,'aggregated_result':aggregated_result,
                             'title':title, 'filter_query_on_borough':filter_query_on_borough}
    
    print('Querying the dabase...')
         
    query_dict = build_query_dict(render_heat_map_dict)
    render_heat_map_dict['query_dict'] = query_dict
    
    
    
    
    
    #we render the heat maps!
    for map_type, base_map, projection in base_maps:
        #we add variabled to the render frame dictionary
        render_heat_map_dict['base_map'] = base_map
        render_heat_map_dict['projection'] = projection
        render_heat_map_dict['map_type'] = map_type
        
        render_heat_map_query_output(render_heat_map_dict)



In [66]:
heat_map_dict = {'shp_path':shp_path, 'image_size':(1920,1080),'db':'nyc_taxi_rides', 
                 'data_table':'taxi_rides_2018','lookup_table':'taxi_zone_lookup_table', 
                 'aggregated_result':'count', 'weekdays_vs_weekends':True,
                 'period':['2018-01-01','2018-01-02'], 'render_single_borough':False,
                  'filter_query_on_borough':False,
                  'map_to_render':['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'],
                  'title':'Title'} 

make_heat_map(heat_map_dict)




Building the base map...
Querying the dabase...
{'data_table': 'taxi_rides_2018', 'lookup_table': 'taxi_zone_lookup_table', 'aggregated_result': 'avg', 'filter_query_on_borough': False, 'date': ['2018-01-01', '2018-01-02'], 'specific_weekdays': 'weekdays_vs_weekends'}
{'data_table': 'taxi_rides_2018', 'lookup_table': 'taxi_zone_lookup_table', 'aggregated_result': 'avg', 'filter_query_on_borough': False, 'date': ['2018-01-01', '2018-01-02'], 'specific_weekdays': 'weekdays_vs_weekends'}
{'data_table': 'taxi_rides_2018', 'lookup_table': 'taxi_zone_lookup_table', 'aggregated_result': 'avg', 'filter_query_on_borough': False, 'date': ['2018-01-01', '2018-01-02'], 'specific_weekdays': 'weekdays_vs_weekends'}
{'data_table': 'taxi_rides_2018', 'lookup_table': 'taxi_zone_lookup_table', 'aggregated_result': 'avg', 'filter_query_on_borough': False, 'date': ['2018-01-01', '2018-01-02'], 'specific_weekdays': 'weekdays_vs_weekends'}
{'data_table': 'taxi_rides_2018', 'lookup_table': 'taxi_zone_lookup_

## Step 5 - render the animations

We are now ready to generate the animations. All the animations generated here are saved with the notebook. 

In general, the common arguments we are going to pass to the function are the following:
- **shp_path** (same shapefile no matter what animation we are rendering)
- **image_size**: [1920, 1080]
- **db**: the nyc_taxi_rides database
- **data_table**: the taxi_rides_2018 table
- **lookup_table**: the taxi_zone_lookup_table table
- **aggregated_result**: count


**Animation 1 - over the year 2018**

This animation is actually composed of a few videos: the flow of passengers everyday in 2018, for the whole city as well as one video per borough.

Let's identify our specific arguments:
- **map_to_render**: for the whole year, we will want to look at the whole city and each borough individually, so we set this to ['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn']
- **render_single_borough**: let's set this to False, so we just focus and zoom on to the borough when rendering them separately
- **filter_query_on_borough**: set to False so we make one query per day that we can reuse to render each animation individually
- **title**: Let's stay simple, with 'General flow of passengers in 2018'. The name of the borough will be appended automatically to the title when rendering the borough-focused animation
- **time_granularity**: we don't want a filter on specific weekdays, so we set this as 'period'
- period: we want the whole year, so ['2018-01-01','2018-12-31']!
- **weekdays**: we want for the whole year, so let's leave this as an empty array.


In [12]:
shp_path = "/Users/acoullandreau/Desktop/Taxi_rides_DS/taxi_zones/taxi_zones.shp"

animation_dict_2018 = {'shp_path':shp_path, 'image_size':(1920,1080), 
                       'map_to_render':['total', 'Manhattan'],
                       'render_single_borough':False,
                       'filter_query_on_borough':False,
                       'title':'General flow of passengers in 2018', 
                       'db':'nyc_taxi_rides', 'data_table':'taxi_rides_2018',
                       'lookup_table':'taxi_zone_lookup_table', 'aggregated_result':'count',
                       'time_granularity':'period', 'period':['2018-01-01','2018-02-02'],  
                       'weekdays':[]}

make_flow_animation(animation_dict_2018)

Building the base map...


Querying the dabase for Manhattan...
Rendering the results for Manhattan...
The video for Manhattan has been rendered


Querying the dabase for Manhattan...


KeyboardInterrupt: 

In [99]:
shp_path = "/Users/acoullandreau/Desktop/Taxi_rides_DS/taxi_zones/taxi_zones.shp"

animation_dict_2018 = {'shp_path':shp_path, 'image_size':(1920,1080), 
                       'map_to_render':['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'],
                       'render_single_borough':False,
                       'filter_query_on_borough':False,
                       'title':'General flow of passengers in 2018', 
                       'db':'nyc_taxi_rides', 'data_table':'taxi_rides_2018',
                       'lookup_table':'taxi_zone_lookup_table', 'aggregated_result':'count',
                       'time_granularity':'period', 'period':['2018-01-01','2018-12-31'],  
                       'weekdays':[]}

In [100]:
make_flow_animation(animation_dict_2018)

KeyboardInterrupt: 

**Animation 2 - over the year 2018, just weekdays**

Same logic, let's create several videos per borough, but this time we want to see only weekdays.


Let's identify our specific arguments:
- **map_to_render**: for the whole year, we will want to look at the whole city and each borough individually, so we set this to ['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn']
- **render_single_borough**: let's set this to False, so we just focus and zoom on to the borough when rendering them separately
- **filter_query_on_borough**: set to False so we make one query per day that we can reuse to render each animation individually
- **title**: Let's stay simple, with 'General flow of passengers on weekdays in 2018'. The name of the borough will be appended automatically to the title when rendering the borough-focused animation
- **time_granularity**: we want a filter on specific weekdays, so we set this as 'on_specific_weekdays'
- period: we want the whole year, so ['2018-01-01','2018-12-31']!
- **weekdays**: we want only for week days, so [0, 1, 2, 3, 4].

In [30]:
animation_dict_weekdays_2018 = {'shp_path':shp_path, 'image_size':(1920,1080), 
                                'map_to_render':['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'],
                                'render_single_borough':False,
                                'filter_query_on_borough':False,
                                'title':'General flow of passengers on weekdays in 2018', 
                                'db':'nyc_taxi_rides', 'data_table':'taxi_rides_2018',
                                'lookup_table':'taxi_zone_lookup_table', 
                                'aggregated_result':'count',
                                'time_granularity':'period', 
                                'period':['2018-01-01','2018-12-31'],  
                                'weekdays':[0, 1, 2, 3, 4]}

In [ ]:
make_flow_animation(animation_dict_weekdays_2018)

**Animation 3 - over the year 2018, just weekends**

Same logic, let's create several videos per borough, but this time we want to see only weekends.



Let's identify our specific arguments:
- **map_to_render**: for the whole year, we will want to look at the whole city and each borough individually, so we set this to ['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn']
- **render_single_borough**: let's set this to False, so we just focus and zoom on to the borough when rendering them separately
- **filter_query_on_borough**: set to False so we make one query per day that we can reuse to render each animation individually
- **title**: Let's stay simple, with 'General flow of passengers on weekends in 2018'. The name of the borough will be appended automatically to the title when rendering the borough-focused animation
- **time_granularity**: we want a filter on specific weekdays, so we set this as 'on_specific_weekdays'
- period: we want the whole year, so ['2018-01-01','2018-12-31']!
- **weekdays**: we want only for weekend days, so [5, 6].

In [29]:
animation_dict_weekends_2018 = {'shp_path':shp_path, 'image_size':(1920,1080), 
                                'map_to_render':['total', 'Manhattan', 'Bronx', 'Queens', 'Staten Island', 'Brooklyn'],
                                'render_single_borough':False,
                                'filter_query_on_borough':False,
                                'title':'General flow of passengers on weekends in 2018', 
                                'db':'nyc_taxi_rides', 'data_table':'taxi_rides_2018',
                                'lookup_table':'taxi_zone_lookup_table', 
                                'aggregated_result':'count',
                                'time_granularity':'period', 
                                'period':['2018-01-01','2018-12, 31'],  
                                'weekdays':[5, 6]}



In [ ]:
make_flow_animation(animation_dict_weekends_2018)

# Step 6 - render the heat maps

In [ ]:
#Create a blank image
blank_image = np.zeros((1080,1920,3), np.uint8) #Size of the image 1080 height, 1920 width, 3 channels of colour
blank_image[:, :] = [255, 255, 255] #Sets the color to white

In [ ]:
#Draws circle ; center, radius, colour, -1 to fill the shape
cv2.circle(blank_image, (960, 540), 200, (0, 255, 0), -1)

In [ ]:
#write text
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(blank_image, 'zone_id', (960, 540), 
            font, 2, (255, 0, 0), 5, cv2.LINE_AA) 
  



In [ ]:
# to show the image inside the Jupyter notebook
plt.imshow(blank_image)
plt.title("White Blank")
plt.show()

In [ ]:
#to show the image outside the notebook
cv2.imshow("White Blank", blank_image) 

**Python script**